# 🍅 Plant Disease Detection with Deep Learning

This project focuses on detecting tomato plant diseases using deep learning and image classification.

- **Dataset**: [PlantVillage Dataset](https://data.mendeley.com/datasets/tywbtsjrjv/1), filtered to include only **tomato leaf images** across various diseases and healthy cases.
- **Objective**: Build a convolutional neural network (CNN) model using **transfer learning** to classify tomato leaves into their respective disease categories or as healthy.
- **Techniques Used**:
  - Data preprocessing and augmentation
  - Transfer learning with pretrained CNNs ( VGG16, MobileNetV2, & InceptionV3 )
  - Model evaluation and accuracy analysis

This project helps demonstrate how deep learning can be applied in agriculture to aid early disease detection and improve crop health monitoring.


In [ ]:
# import necessary modules

import numpy as np
import gradio as gr

from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array
from tensorflow.keras.applications import VGG16, MobileNetV2, InceptionV3
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten
# from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

In [2]:
# build data generator for training

# normalize image and carryout data generation
train_datagen = ImageDataGenerator(rescale=1./255,          # normalize pixel values        
                               rotation_range=20,       # randomly rotate images by up to 20 degrees
                               zoom_range=0.2,          # randomly zoom images
                               horizontal_flip=True     # randomly flip images horizontally
                               )

# define data generator for validation and test sets
datagen = ImageDataGenerator(rescale=1./255)

# load training, validation and testing images from directory
train_data = train_datagen.flow_from_directory("PlantVillage_Tomato_Split/train",    # file path
                                              target_size=(224, 224),               # resize images to fit transfer model input
                                              batch_size=32,                         # process 32 images at a time
                                              class_mode="categorical"              # one hot encode labels
                                              )

val_data = datagen.flow_from_directory("PlantVillage_Tomato_Split/val",
                                        target_size=(224, 224),
                                        batch_size=32,
                                        class_mode="categorical"
                                        )

test_data = datagen.flow_from_directory("PlantVillage_Tomato_Split/test",
                                        target_size=(225, 225),
                                        batch_size=32,
                                        class_mode="categorical",
                                        shuffle=False                   # do not shuffle testing data
                                        )

Found 12707 images belonging to 10 classes.
Found 3628 images belonging to 10 classes.
Found 1825 images belonging to 10 classes.


## VGG16 model

In [3]:
# define model architecture
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation="relu")(x)
predictions = Dense(10, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=predictions)

# freeze base model
for layer in base_model.layers:
    layer.trainable = False

In [4]:
# check model summary
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [5]:
# compile model
model.compile(optimizer=Adam(0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# define early stoppage parameters
#early_stopping = EarlyStopping(monitor="val_loss", patience=8)

# fit model on training data. set epoch to 5
model.fit(train_data, validation_data=val_data, epochs=5)

# uncomment for more epochs and early stopping
#model.fit(train_data, validation_data=val_data, epochs=100, callbacks=[early_stopping])

Epoch 1/5
398/398 [==============================] - 267s 640ms/step - loss: 1.9189 - accuracy: 0.3288 - val_loss: 1.6967 - val_accuracy: 0.4220
Epoch 2/5
398/398 [==============================] - 178s 447ms/step - loss: 1.5418 - accuracy: 0.5389 - val_loss: 1.4255 - val_accuracy: 0.5739
Epoch 3/5
398/398 [==============================] - 180s 452ms/step - loss: 1.3214 - accuracy: 0.6171 - val_loss: 1.2395 - val_accuracy: 0.6329
Epoch 4/5
398/398 [==============================] - 266s 668ms/step - loss: 1.1718 - accuracy: 0.6563 - val_loss: 1.1236 - val_accuracy: 0.6590
Epoch 5/5
398/398 [==============================] - 364s 908ms/step - loss: 1.0641 - accuracy: 0.6802 - val_loss: 1.0143 - val_accuracy: 0.6932


In [6]:
loss, accuracy = model.evaluate(test_data)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

58/58 [==============================] - 14s 216ms/step - loss: 1.0292 - accuracy: 0.6964
Test Accuracy: 69.64%


### Make a few layers trainable

In [7]:
# define model architecture
base_model1 = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
x = base_model1.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation="relu")(x)
predictions = Dense(10, activation="softmax")(x)

vgg16_model2 = Model(inputs=base_model1.input, outputs=predictions)

# freeze base model, making onlt last two trainable
for layer in base_model1.layers:
    layer.trainable = False

for layer in base_model1.layers[-2:]:
    layer.trainable = True

In [8]:
vgg16_model2.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [9]:
# compile model
vgg16_model2.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# define early stoppage parameters
#early_stopping = EarlyStopping(monitor="val_loss", patience=8)

# fit model on training data. set epoch to 5
vgg16_model2.fit(train_data, validation_data=val_data, epochs=5)

# uncomment for more epochs and early stopping
#model.fit(train_data, validation_data=val_data, epochs=100, callbacks=[early_stopping])

Epoch 1/5
398/398 [==============================] - 169s 421ms/step - loss: 0.8027 - accuracy: 0.7286 - val_loss: 0.5588 - val_accuracy: 0.8024
Epoch 2/5
398/398 [==============================] - 165s 414ms/step - loss: 0.4335 - accuracy: 0.8521 - val_loss: 0.6056 - val_accuracy: 0.7999
Epoch 3/5
398/398 [==============================] - 177s 444ms/step - loss: 0.3518 - accuracy: 0.8781 - val_loss: 0.4363 - val_accuracy: 0.8495
Epoch 4/5
398/398 [==============================] - 169s 424ms/step - loss: 0.2863 - accuracy: 0.9008 - val_loss: 0.5239 - val_accuracy: 0.8264
Epoch 5/5
398/398 [==============================] - 205s 516ms/step - loss: 0.2436 - accuracy: 0.9154 - val_loss: 0.3184 - val_accuracy: 0.8889


In [10]:
loss, accuracy = vgg16_model2.evaluate(test_data)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


58/58 [==============================] - 7s 108ms/step - loss: 0.3119 - accuracy: 0.8997
Test Accuracy: 89.97%


## MobileNet model

In [11]:
# define model architecture
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation="relu")(x)
predictions = Dense(10, activation="softmax")(x)

mobilenet_model1 = Model(inputs=base_model.input, outputs=predictions)

# freeze base model
for layer in base_model.layers:
    layer.trainable = False


In [12]:
mobilenet_model1.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                           

In [13]:
# compile model
mobilenet_model1.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# define early stoppage parameters
#early_stopping = EarlyStopping(monitor="val_loss", patience=8)

# fit model on training data. set epoch to 5
mobilenet_model1.fit(train_data, validation_data=val_data, epochs=5)

# uncomment for more epochs and early stopping
#model.fit(train_data, validation_data=val_data, epochs=100, callbacks=[early_stopping])

Epoch 1/5
398/398 [==============================] - 137s 339ms/step - loss: 0.5854 - accuracy: 0.8040 - val_loss: 0.5043 - val_accuracy: 0.8299
Epoch 2/5
398/398 [==============================] - 155s 390ms/step - loss: 0.3570 - accuracy: 0.8772 - val_loss: 0.3871 - val_accuracy: 0.8696
Epoch 3/5
398/398 [==============================] - 102s 256ms/step - loss: 0.2894 - accuracy: 0.9002 - val_loss: 0.4565 - val_accuracy: 0.8545
Epoch 4/5
398/398 [==============================] - 98s 246ms/step - loss: 0.2566 - accuracy: 0.9126 - val_loss: 0.3593 - val_accuracy: 0.8823
Epoch 5/5
398/398 [==============================] - 91s 227ms/step - loss: 0.2318 - accuracy: 0.9195 - val_loss: 0.3131 - val_accuracy: 0.8997


In [14]:
loss, accuracy = mobilenet_model1.evaluate(test_data)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

58/58 [==============================] - 2s 27ms/step - loss: 0.4001 - accuracy: 0.8548
Test Accuracy: 85.48%


### Make a few layers trainable

In [83]:
# define model architecture
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
x = base_model.output
# apply global average pooling 
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation="relu")(x)
predictions = Dense(10, activation="softmax")(x)

mobilenet_model2 = Model(inputs=base_model.input, outputs=predictions)

# freeze layers of base model except last two
for layer in base_model.layers:
    layer.trainable = False

for layer in base_model.layers[-3:]:
    layer.trainable = True

In [84]:
mobilenet_model2.summary()

Model: "model_30"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_32 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_32[0][0]']               
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                          

In [53]:
# compile model
mobilenet_model2.compile(loss="categorical_crossentropy", metrics=["Accuracy"], optimizer=Adam(0.001))

# fit model to data
mobilenet_model2.fit(train_data, validation_data=val_data, epochs=5)

Epoch 1/5
398/398 [==============================] - 97s 239ms/step - loss: 0.5757 - Accuracy: 0.8063 - val_loss: 0.6878 - val_Accuracy: 0.7767
Epoch 2/5
398/398 [==============================] - 127s 319ms/step - loss: 0.3642 - Accuracy: 0.8774 - val_loss: 0.6456 - val_Accuracy: 0.7897
Epoch 3/5
398/398 [==============================] - 216s 541ms/step - loss: 0.2881 - Accuracy: 0.8994 - val_loss: 0.4408 - val_Accuracy: 0.8545
Epoch 4/5
398/398 [==============================] - 174s 437ms/step - loss: 0.2615 - Accuracy: 0.9093 - val_loss: 0.4582 - val_Accuracy: 0.8523
Epoch 5/5
398/398 [==============================] - 82s 205ms/step - loss: 0.2291 - Accuracy: 0.9211 - val_loss: 0.2931 - val_Accuracy: 0.9030


In [54]:
loss, accuracy = mobilenet_model2.evaluate(test_data)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

58/58 [==============================] - 2s 26ms/step - loss: 0.3732 - Accuracy: 0.8805
Test Accuracy: 88.05%


## InceptionV3 Models

In [55]:
# define model architecture
base_model = InceptionV3(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation="relu")(x)
predictions = Dense(10, activation="softmax")(x)

inception_model = Model(inputs=base_model.input, outputs=predictions)
# freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

87910968/87910968 [==============================] - 30s 0us/step


In [56]:
inception_model.summary()

Model: "model_17"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_19 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_19[0][0]']               
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                          

In [ ]:
# compile model
inception_model.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# define early stoppage parameters
# early_stopping = EarlyStopping(monitor="val_loss", patience=8)

# fit model on training data. set epoch to 5
inception_model.fit(train_data, validation_data=val_data, epochs=5)

# uncomment for more epochs and early stopping
# inception_model.fit(train_data, validation_data=val_data, epochs=100, callbacks=[early_stopping])

Epoch 1/5
398/398 [==============================] - 168s 415ms/step - loss: 0.8247 - accuracy: 0.7203 - val_loss: 0.5889 - val_accuracy: 0.8024
Epoch 2/5
398/398 [==============================] - 150s 375ms/step - loss: 0.5542 - accuracy: 0.8101 - val_loss: 0.5413 - val_accuracy: 0.8087
Epoch 3/5
398/398 [==============================] - 166s 417ms/step - loss: 0.4879 - accuracy: 0.8291 - val_loss: 0.6657 - val_accuracy: 0.7715
Epoch 4/5
398/398 [==============================] - 93s 233ms/step - loss: 0.4659 - accuracy: 0.8413 - val_loss: 0.4692 - val_accuracy: 0.8401
Epoch 5/5
398/398 [==============================] - 90s 227ms/step - loss: 0.4010 - accuracy: 0.8604 - val_loss: 0.4770 - val_accuracy: 0.8440


In [58]:
loss, accuracy = inception_model.evaluate(test_data)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

58/58 [==============================] - 3s 53ms/step - loss: 0.4882 - accuracy: 0.8455
Test Accuracy: 84.55%


### Make a few layers trainable

In [ ]:
# define model architecture
base_model = InceptionV3(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation="relu")(x)
predictions = Dense(10, activation="softmax")(x)

inception_model2 = Model(inputs=base_model.input, outputs=predictions)
# freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# make last 15 base layers trainable
for layer in base_model.layers[-15:]:
    layer.trainable = True

In [76]:
inception_model2.summary()

Model: "model_26"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_28 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_846 (Conv2D)            (None, 111, 111, 32  864         ['input_28[0][0]']               
                                )                                                                 
                                                                                                  
 batch_normalization_846 (Batch  (None, 111, 111, 32  96         ['conv2d_846[0][0]']             
 Normalization)                 )                                                          

In [ ]:
# compile model
inception_model2.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# define early stoppage parameters
# early_stopping = EarlyStopping(monitor="val_loss", patience=8)

# fit model on training data. set epoch to 5
inception_model2.fit(train_data, validation_data=val_data, epochs=5)

# uncomment for more epochs and early stopping
# inception_model2.fit(train_data, validation_data=val_data, epochs=100, callbacks=[early_stopping])

Epoch 1/5
398/398 [==============================] - 95s 231ms/step - loss: 0.6088 - accuracy: 0.7994 - val_loss: 0.3743 - val_accuracy: 0.8740
Epoch 2/5
398/398 [==============================] - 103s 258ms/step - loss: 0.3382 - accuracy: 0.8830 - val_loss: 0.3443 - val_accuracy: 0.8784
Epoch 3/5
398/398 [==============================] - 128s 321ms/step - loss: 0.2700 - accuracy: 0.9065 - val_loss: 0.3155 - val_accuracy: 0.8969
Epoch 4/5
398/398 [==============================] - 86s 217ms/step - loss: 0.2424 - accuracy: 0.9153 - val_loss: 0.2430 - val_accuracy: 0.9123
Epoch 5/5
398/398 [==============================] - 92s 230ms/step - loss: 0.2247 - accuracy: 0.9216 - val_loss: 0.2240 - val_accuracy: 0.9176


In [86]:
loss, accuracy = inception_model2.evaluate(test_data)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

58/58 [==============================] - 3s 51ms/step - loss: 0.2444 - accuracy: 0.9178
Test Accuracy: 91.78%


## Model Deployment

In [ ]:
# save inception model
inception_model2.save("tomato_disease_inception_model.keras")

In [ ]:
# Load trained model
model = load_model("tomato_disease_inception_model.keras")
# access class labels
class_labels = list(train_data.class_indices.keys())
print(class_labels)

In [ ]:
def predict(img):
    """ This function preprocesses the image received as input and carries out model predictions, 
        returning the predicted class and model confidence from prediction.
    """
    # resize image, convert it to and array and normalize it
    img = img.resize((224, 224))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    predictions = model.predict(img_array)
    predicted_class = class_labels[np.argmax(predictions)]
    confidence = np.max(predictions)
    return f"{predicted_class} ({confidence:.2%})"

In [ ]:
gr.Interface(fn=predict, 
             inputs=gr.Image(type="pil"), 
             outputs="text").launch()